In [1]:
import pandas as pd
import os
import sys
import datetime
import time
import re
from sqlalchemy import create_engine
from sqlalchemy import text
import sqlalchemy


In [3]:
# get brand_id-cartoon-product_id relationship

In [4]:
conn_string = os.environ['PG_DB2']
db = create_engine(conn_string.replace('multdb', 'youtube'))

In [5]:
conn = db.connect()
req1 = '''select distinct(brand_id),cartoon, product_id from intl_yt_reels
where cartoon in
(select distinct(cartoon) from intl_yt_reels
where brand_id != '')
order by brand_id'''
cartoons = pd.read_sql(text(req1), conn)
conn.close()

In [8]:
df_in = pd.read_excel('markuped+multiclass_new.xlsx')

In [13]:
brands2add = df_in[0].unique()

In [57]:
def update_db(ids, cartoon, brand_id, product_id):
    conn = db.connect()
    print(cartoon, end = ':')
    ids_str = "('" +  "','".join(to_add_ids) + "')"
    req = '''UPDATE intl_yt_reels set cartoon = '{cartoon}', brand_id = '{brand_id}', product_id = '{product_id}' 
    where yt_reel_id in {ids_str}'''.format(cartoon = cartoon, brand_id = brand_id, product_id = product_id, ids_str = ids_str)
    rs = conn.execute(text(req))
    print(rs.rowcount)
    conn.commit()
    conn.close()

In [58]:
for brand in brands2add:
    to_add_ids = df_in[df_in[0] == brand].yt_reel_id.to_list()
    cartoon = cartoons[cartoons.brand_id == brand].cartoon.values[0]
    product_id = cartoons[cartoons.brand_id == brand].product_id.values[0]
    update_db(to_add_ids, cartoon, brand, product_id)

Енотки:13
Ну, погоди! Каникулы:2
Умка:2
Оранжевая корова:13
Пластилинки:1
Приключения Пети и волка:7


In [1]:
#df_in